In [ ]:
from pathlib import Path
import numpy as np

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table

from astroquery.gaia import Gaia

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def two_up(B, V, title, mag_col='mag_inst_cal'):
    fig, ax = plt.subplots(1, 2, figsize=(20, 10))
    resid, cmd = ax
    resid.plot(B['color_cat'], B['color_cat'] - (B[mag_col] - V[mag_col]), '.', label='catalog colors', alpha=0.4)
    #plt.ylim(16.5, 10)
    resid.set_xlabel('Catalog B-V')
    resid.set_ylabel('Diff between catalog and calibrated B-V')
    resid.set_title(title)
    resid.grid()
    good_color = np.abs(B['color_cat'] - (B['mag_inst_cal'] - V['mag_inst_cal'])) < 0.1
    
    cmd.plot(B[mag_col][good_color] - V[mag_col][good_color], V[mag_col][good_color], '.')
    cmd.set_ylim(*cmd.get_ylim()[::-1])
    cmd.set_ylabel('V')
    cmd.set_xlabel('B - V')
    cmd.set_xlim(-0.5, 2.5)
    cmd.grid()
    

In [ ]:
your_mag_table = 'donovan-photo_m1.csv'

In [ ]:
mag_table = Table.read(your_mag_table)

In [ ]:
B = mag_table[mag_table['filter'] == 'B']
V = mag_table[mag_table['filter'] == 'V']

In [ ]:
mag_col = 'mag_inst_cal'
fig, resid = plt.subplots(1, 1, figsize=(10, 10))

resid.plot(B['color_cat'], B['color_cat'] - (B[mag_col] - V[mag_col]), '.', label='catalog colors', alpha=0.4)
#plt.ylim(16.5, 10)
resid.set_xlabel('Catalog B-V')
resid.set_ylabel('Diff between catalog and calibrated B-V')
resid.set_title(your_mag_table)
resid.grid()

In [ ]:
good_color = np.abs(B['color_cat'] - (B['mag_inst_cal'] - V['mag_inst_cal'])) < 0.1

In [ ]:
fig, cmd = plt.subplots(1, 1, figsize=(10, 10))

cmd.plot(B[mag_col][good_color] - V[mag_col][good_color], V[mag_col][good_color], '.')
cmd.set_ylim(*cmd.get_ylim()[::-1])
cmd.set_ylabel('V')
cmd.set_xlabel('B - V')
cmd.set_xlim(-0.5, 2.5)
cmd.grid()


In [ ]:
sc = SkyCoord(ra=mag_table['RA'].mean(), dec=mag_table['Dec'].mean(), unit='degree')

In [ ]:
Gaia.ROW_LIMIT = -1
j = Gaia.cone_search_async(sc, 20 * u.arcmin)

In [ ]:
r = j.get_results()

In [ ]:
our_cs = SkyCoord(ra=mag_table['RA'], dec=mag_table['Dec'], unit='degree')

In [ ]:
gaia_cs = SkyCoord(ra=r['ra'], dec=r['dec'], unit='degree')

In [ ]:
idx, d2d, _ = our_cs.match_to_catalog_sky(gaia_cs)

In [ ]:
great = d2d < 1 * u.arcsec

In [ ]:
distance = 1000 / r['parallax']

In [ ]:
abs_mag = 0 * mag_table['RA']

In [ ]:
abs_mag[great] = mag_table['mag_inst_cal'][great] - 5 * (np.log10(distance[idx[great]]) - 1)

In [ ]:
mag_table['abs_mag'] = abs_mag

In [ ]:
B = mag_table[mag_table['filter'] == 'B']
V = mag_table[mag_table['filter'] == 'V']
good = (B['abs_mag'] != 0) & (V['abs_mag'] != 0) & ~np.isnan(B['abs_mag']) & ~np.isnan(V['abs_mag'])
two_up(B[good], V[good], your_mag_table, mag_col='abs_mag')